In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Using Language Model


In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fcc4ae99-a5f5-40e3-baf1-0dbb89e930b8-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

# OutputParsers

The response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.


### Simple Usage

We could save the result of the language model call and then pass it to the parser.


In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)

parser.invoke(result)

'Ciao!'

### More Common Usage

We can "chain" the model with this output parser. This means this output parser will get called every time in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).


In [5]:
chain = model | parser

chain.invoke(messages)

'Ciao!'

The above is a simple example of a chain. It has two steps: 1. the language model is called 2. the result is passed to the output parser.
The `|` operator is used in LangChain to combine two elements together.


# Prompt Templates

`PromptTemplates` are a concept in LangChain designed to assist with the translation of raw user input into a format that can be passed into a language model.

They take in raw user input and return data (a prompt) that is ready to pass into a language model.


For example, the following PromptTemplate takes in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate


In [6]:
from langchain_core.prompts import ChatPromptTemplate


system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

ChatPromptValue consists of two messages: 1. system message 2. user message. If we only need the `messages`, we can use `to_messages()`.


In [7]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

Now, we can chain the prompt template with the language model and output parser to create a complete pipeline.


In [8]:
chain = prompt_template | model | parser

In [9]:
chain.invoke({"language": "italian", "text": "hi"})

'Ciao'